In [72]:
'''
作者：英俊
QQ:2227495940
邮箱 同上
'''

'\n作者：英俊\nQQ:2227495940\n邮箱 同上\n'

In [11]:
#导入Pytorch包

import torch

import torch.nn as nn

from fastNLP.io.loader import CSVLoader

dataset_loader = CSVLoader(headers=('raw_words','target'), sep='\t')


In [12]:
#看来分隔符很重要啊
testset_loader = CSVLoader( headers=['raw_words'],sep='\t')

In [13]:
# 表示将CSV文件中每一行的第一项将填入'raw_words' field，第二项填入'target' field。

# 其中项之间由'\t'分割开来

train_path=r'train_shuffle.txt'

test_path=r'new_test_handout.txt'

dataset = dataset_loader._load(train_path)

testset = testset_loader._load(test_path)


In [14]:
# 将句子分成单词形式, 详见DataSet.apply()方法

import jieba

from itertools import chain

print(jieba.__version__)
# from itertools import chain

#     '''

#     @params:

#         data: 数据的列表，列表中的每个元素为 [文本字符串，0/1标签] 二元组

#     @return: 切分词后的文本的列表，列表中的每个元素为切分后的词序列

#     '''

def get_tokenized(data,words=True):
    def tokenizer(text):
        return [tok for tok in jieba.cut(text, cut_all=False)]
    if words:

        #按词语进行编码

        return tokenizer(data)

    else:

        #按字进行编码

        return [tokenizer(review) for review in data]


0.39


In [15]:
print(dataset)

+-------------------------------------+--------+
| raw_words                           | target |
+-------------------------------------+--------+
| 那是什么破酒店，你们携程也对外推... | 0      |
| 非常一般的酒店，服务台上挂的特色... | 0      |
| 入住了传说中的04房间，景色真的很... | 1      |
| 餐厅很差,菜的种类水准都不行.        | 0      |
|                                     |        |
| 酒...                               |        |
| 酒店位于福州最热闹的东街口，出来... | 1      |
| 上月入住，将近500元一天的房费，...  | 0      |
| 老掉牙的五星酒店，我们入住的客房... | 0      |
| 酒店位置在很市中心的地方,房间也...  | 1      |
| 除了停车不是很方便，其他的都还不... | 1      |
| 原先看了别人的评价，做好了不怎么... | 1      |
| 宜宾还有很多好的酒店，比如说叙府... | 0      |
| 不错,变成常驻京酒店了,就是餐厅的... | 1      |
| 最近变成了四星，好像房间装修过了... | 0      |
| 一分钟一块钱的网费，贵的惊人。三... | 0      |
| 四个字，“糟糕透顶”。我是晚上十...   | 0      |
| 我住的是大床房，房间很大，还有一... | 1      |
| 几个月之后再次入住，一进门大厅里... | 1      |
| 这次和老公入住这家酒店 640元的豪... | 0      |
| 忘了说，前台服务较烂，中午在莫泰... | 0      |
| 房间价格奇贵，上网费2元每分钟，...  | 0      |
| 不错，离大连港很近，步行几分钟就... | 1      |
| 前台的服务很差，10月1日的值班经...  | 0      |
| 酒店邻近山脚，空气清新，闹

In [18]:
dataset.apply(lambda ins:get_tokenized(ins['raw_words']), new_field_name='words', is_input=True)
print(dataset)

+----------------------+--------+----------------------+----------------------+
| raw_words            | target | chars                | words                |
+----------------------+--------+----------------------+----------------------+
| 那是什么破酒店，...  | 0      | ['那', '是', '什...  | ['那', '是', '什...  |
| 非常一般的酒店，...  | 0      | ['非常', '一般',...  | ['非常', '一般',...  |
| 入住了传说中的04...  | 1      | ['入住', '了', '...  | ['入住', '了', '...  |
| 餐厅很差,菜的种类... | 0      | ['餐厅', '很差',...  | ['餐厅', '很差',...  |
| 酒店位于福州最热...  | 1      | ['酒店', '位于',...  | ['酒店', '位于',...  |
| 上月入住，将近50...  | 0      | ['上', '月', '入...  | ['上', '月', '入...  |
| 老掉牙的五星酒店...  | 0      | ['老掉牙', '的',...  | ['老掉牙', '的',...  |
| 酒店位置在很市中...  | 1      | ['酒店', '位置',...  | ['酒店', '位置',...  |
| 除了停车不是很方...  | 1      | ['除了', '停车',...  | ['除了', '停车',...  |
| 原先看了别人的评...  | 1      | ['原先', '看', '...  | ['原先', '看', '...  |
| 宜宾还有很多好的...  | 0      | ['宜宾', '还有',...  | ['宜宾', '还有',...  |
| 不错,变成常驻京酒... | 1      | ['不错', ',', '变... | 

In [20]:
dataset.apply(lambda ins: len(ins['words']) ,new_field_name='seq_len', is_input=True)

print(dataset)

+-------------------+--------+-------------------+-------------------+---------+
| raw_words         | target | chars             | words             | seq_len |
+-------------------+--------+-------------------+-------------------+---------+
| 那是什么破酒店... | 0      | ['那', '是', ...  | ['那', '是', ...  | 330     |
| 非常一般的酒店... | 0      | ['非常', '一般... | ['非常', '一般... | 36      |
| 入住了传说中的... | 1      | ['入住', '了'...  | ['入住', '了'...  | 69      |
| 餐厅很差,菜的...  | 0      | ['餐厅', '很差... | ['餐厅', '很差... | 35      |
| 酒店位于福州最... | 1      | ['酒店', '位于... | ['酒店', '位于... | 81      |
| 上月入住，将近... | 0      | ['上', '月', ...  | ['上', '月', ...  | 98      |
| 老掉牙的五星酒... | 0      | ['老掉牙', '的... | ['老掉牙', '的... | 42      |
| 酒店位置在很市... | 1      | ['酒店', '位置... | ['酒店', '位置... | 25      |
| 除了停车不是很... | 1      | ['除了', '停车... | ['除了', '停车... | 30      |
| 原先看了别人的... | 1      | ['原先', '看'...  | ['原先', '看'...  | 235     |
| 宜宾还有很多好... | 0      | ['宜宾', '还有... | ['宜宾', '还有... | 22      |
| 不错,变成常驻...  | 1 

In [21]:
dataset.apply(lambda x: int(x['target']), new_field_name='target', is_target=True)
print(dataset)

+-------------------+--------+-------------------+-------------------+---------+
| raw_words         | target | chars             | words             | seq_len |
+-------------------+--------+-------------------+-------------------+---------+
| 那是什么破酒店... | 0      | ['那', '是', ...  | ['那', '是', ...  | 330     |
| 非常一般的酒店... | 0      | ['非常', '一般... | ['非常', '一般... | 36      |
| 入住了传说中的... | 1      | ['入住', '了'...  | ['入住', '了'...  | 69      |
| 餐厅很差,菜的...  | 0      | ['餐厅', '很差... | ['餐厅', '很差... | 35      |
| 酒店位于福州最... | 1      | ['酒店', '位于... | ['酒店', '位于... | 81      |
| 上月入住，将近... | 0      | ['上', '月', ...  | ['上', '月', ...  | 98      |
| 老掉牙的五星酒... | 0      | ['老掉牙', '的... | ['老掉牙', '的... | 42      |
| 酒店位置在很市... | 1      | ['酒店', '位置... | ['酒店', '位置... | 25      |
| 除了停车不是很... | 1      | ['除了', '停车... | ['除了', '停车... | 30      |
| 原先看了别人的... | 1      | ['原先', '看'...  | ['原先', '看'...  | 235     |
| 宜宾还有很多好... | 0      | ['宜宾', '还有... | ['宜宾', '还有... | 22      |
| 不错,变成常驻...  | 1 

In [22]:
#testset.apply(lambda ins: list(chain.from_iterable(get_tokenized(ins['raw_words']))), new_field_name='words', is_input=True)

testset.apply(lambda ins: get_tokenized(ins['raw_words']), new_field_name='words', is_input=True)

testset.apply(lambda ins: len(ins['words']) ,new_field_name='seq_len',is_input=True)
print(testset)

+---------------------------+---------------------------+---------+
| raw_words                 | words                     | seq_len |
+---------------------------+---------------------------+---------+
| 入住大连富丽华东楼，新... | ['入住', '大连', '富丽... | 17      |
| 6月初入住的，据说后楼...  | ['6', '月初', '入住',...  | 161     |
| 三人间，房间很宽敞，卫... | ['三人间', '，', '房间... | 76      |
| 很好．非常干净．是新装... | ['很', '好', '．', '非... | 29      |
| 4月份回青岛顺便带朋友...  | ['4', '月份', '回', '...  | 66      |
| 我是看了携程的评价才去... | ['我', '是', '看', '了... | 138     |
| 早上八点多到酒店,预先...  | ['早上', '八点', '多到... | 15      |
| 该酒店根本就达不到星级... | ['该', '酒店', '根本'...  | 80      |
| 订的单人间，结果没宽带... | ['订', '的', '单人间'...  | 52      |
| 差的不能在差了，厕所垃... | ['差', '的', '不能', ...  | 40      |
| 潮州迎宾馆确实装修了，... | ['潮州', '迎宾馆', '确... | 49      |
| 我订了二个房间。酒店隔... | ['我订', '了', '二个'...  | 199     |
| 出于不信邪的原因，入住... | ['出于', '不信邪', '的... | 79      |
| 我订的是430元的大床间...  | ['我订', '的', '是', ...  | 285     |
| 前台服務員的禮貌有待改... | ['前台', '服務員', '的... | 97      |
| 入住的5天，天天

In [23]:
###

from fastNLP import Vocabulary

#将DataSet按照ratio的比例拆分，返回两个DataSet

#ratio (float) -- 0<ratio<1, 返回的第一个DataSet拥有 (1-ratio) 这么多数据，第二个DataSet拥有`ratio`这么多数据

train_data, dev_data = dataset.split(0.1, shuffle=False)


In [24]:
print(train_data)

+-------------------+--------+-------------------+-------------------+---------+
| raw_words         | target | chars             | words             | seq_len |
+-------------------+--------+-------------------+-------------------+---------+
| 这是我见过最垃... | 0      | ['这', '是', ...  | ['这', '是', ...  | 136     |
| 设施很陈旧，卫... | 0      | ['设施', '很'...  | ['设施', '很'...  | 12      |
| 携程的员工最好... | 0      | ['携程', '的'...  | ['携程', '的'...  | 158     |
| 很好，这次入住... | 1      | ['很', '好', ...  | ['很', '好', ...  | 93      |
| 因为春节参加朋... | 0      | ['因为', '春节... | ['因为', '春节... | 74      |
| 前台服务较差，... | 1      | ['前台', '服务... | ['前台', '服务... | 32      |
| 比较老的酒店，... | 1      | ['比较', '老'...  | ['比较', '老'...  | 20      |
| 我住的是大床间... | 0      | ['我', '住', ...  | ['我', '住', ...  | 40      |
| 我是在海洋公园... | 1      | ['我', '是', ...  | ['我', '是', ...  | 355     |
| 回来2天了，很...  | 1      | ['回来', '2',...  | ['回来', '2',...  | 724     |
| 性价比低，感觉... | 0      | ['性价比', '低... | ['性价比', '低... | 47      |
| 住宿

In [25]:
print(len(train_data),len(dev_data),len(testset))

1080 120 800


In [26]:
vocab = Vocabulary(min_freq=2).from_dataset(dataset, field_name='words')

vocab.index_dataset(train_data, dev_data, testset, field_name='words', new_field_name='words')

Vocabulary(['那', '是', '什么', '破', '酒店']...)

In [27]:
from fastNLP.embeddings import StaticEmbedding,StackEmbedding

fastnlp_embed = StaticEmbedding(vocab, model_dir_or_name='cn-char-fastnlp-100d',min_freq=2)


Found 709 out of 5160 words in the pre-training embedding.


In [28]:
from fastNLP.models import CNNText

model_CNN = CNNText(fastnlp_embed, num_classes=2,dropout=0.1)

print(model_CNN)


CNNText(
  (embed): Embedding(
    (embed): StaticEmbedding(
      (dropout_layer): Dropout(p=0)
      (embedding): Embedding(5160, 100, padding_idx=0)
    )
    (dropout): Dropout(p=0.0)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(100, 30, kernel_size=(1,), stride=(1,), bias=False)
      (1): Conv1d(100, 40, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (2): Conv1d(100, 50, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    )
  )
  (dropout): Dropout(p=0.1)
  (fc): Linear(in_features=120, out_features=2, bias=True)
)


In [30]:
from fastNLP import Trainer, CrossEntropyLoss, AccuracyMetric,BCELoss

trainer_CNN = Trainer(model=model_CNN, train_data=train_data, dev_data=dev_data,loss=CrossEntropyLoss(), metrics=AccuracyMetric())

trainer_CNN.train()


input fields after batch(if batch size is 2):
	chars: (1)type:numpy.ndarray (2)dtype:object, (3)shape:(2,) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 136]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2020-06-18-10-42-05-446458


Evaluate data in 0.6 seconds!
Evaluation on dev at Epoch 1/10. Step:34/340: 
AccuracyMetric: acc=0.833333



Evaluate data in 0.63 seconds!
Evaluation on dev at Epoch 2/10. Step:68/340: 
AccuracyMetric: acc=0.9



Evaluate data in 0.59 seconds!
Evaluation on dev at Epoch 3/10. Step:102/340: 
AccuracyMetric: acc=0.858333



Evaluate data in 0.61 seconds!
Evaluation on dev at Epoch 4/10. Step:136/340: 
AccuracyMetric: acc=0.925



...

Evaluate data in 0.68 seconds!
Evaluation on dev at Epoch 7/10. Step:238/340: 
AccuracyMetric: acc=0.908333



Evaluate data in 0.76 seconds!
Evaluation on dev at Epoch 8/10. Step:272/340: 
AccuracyMetric: acc=0.908333



Evaluate data in 0.58 seconds!
Evaluation on dev at Epoch 9/10. Step:306/340: 
AccuracyMetric: acc=0.916667



Evaluate data in 0.63 seconds!
Evaluation on dev at Epoch 10/10. Step:340/340: 
AccuracyMetric: acc=0.908333

Reloaded the best model.

In Epoch:4/Step:136, got best dev performance:
AccuracyMetric: acc=0.925


{'best_epoch': 4,
 'best_eval': {'AccuracyMetric': {'acc': 0.925}},
 'best_step': 136,
 'seconds': 124.97}

In [60]:
demo=[]

In [70]:
#批量进行数据预测

import pandas as pd

import torch

def batch_predict(model,data):
    submission=pd.DataFrame(columns=['Prediction'])
#    submission = pd.DataFrame(columns=['ID','Prediction'])
    for i in range(len(data)):
    #for i in range(5):
#         print(data.words[i])
        tensor = torch.tensor(data.words[i])
        pred = model.predict(tensor.view(1,-1))
#         print(pred)
        prob = pred['pred'].numpy()[0]
#         print("pred:%.2f"%(prob))
#         print('='*50)
#         print(type(prob))
        s2 = pd.Series([float(prob)], index=['Prediction'])
        demo.append(prob)
#         print(s2)
        submission = submission.append(s2, ignore_index=True)
        submission['Prediction'] = submission.Prediction .astype(int)
#         submission['']
#         submission['Prediction'] = submission.Prediction.astype(float) 
    return submission


In [71]:
#开始进行预测，并将结果保存到提交格式文件中，提交平台

# summission_path = r'data\Comments9120'

submission = batch_predict(model_CNN,testset)
submission.to_csv('fastnlpDemo.csv',encoding='utf-8')
# submission.to_csv(summission_path+'\submission-CNN-20200229-words.csv', index=False)
